# One-dimensional case

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
dataset = smooth.datasets.GaussianProcessDataset(samples_train=10, lengthscale=0.3, noise_var=0.0, seed=1)

In [ ]:
model = smooth.model.train_shallow(
    dataset,
    learning_rate=0.01,
    init_scale=0.1,
    epochs=10000,
    verbose=0,
    batch_size=64,
)

In [ ]:
smooth.analysisysisysisysisalysis.plot

In [ ]:
smooth.measures.path_length(model, dataset.x_train, derivative=True, segments=10)

In [ ]:
model.predict(dataset.x_train[:10])

In [ ]:
smooth.measures.get_measures(model, dataset, precise_in_1d=False)